# Gépi Tanulás Klasszifikációs Feladatok Összefoglalója

Ez a notebook összefoglalja a két klasszifikációs feladatot:
1. Osztályozás szintetikus adatokon.
2. Osztályozás a Wine adathalmazon.

*Megjegyzés: A kódcellák csak a felhasznált kulcsfontosságú függvényeket és koncepciókat szemléltetik, nem a teljes futtatható kódot.*

## Szükséges Könyvtárak Importálása

Először importáljuk az összes szükséges könyvtárat mindkét feladathoz.

In [ ]:
import numpy as np
import pandas as pd
import time
import os
import warnings
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.exceptions import ConvergenceWarning
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Feladat: Osztályozás Szintetikus Adatokon

## 1.1 Adatgenerálás

Létrehozunk egy mesterséges adathalmazt `make_blobs` segítségével.

In [ ]:
X_part1, y_part1 = make_blobs(n_samples=..., n_features=..., centers=..., random_state=...)

## 1.2 Adat Vizualizációja (PCA)

Főkomponens Analízist (PCA) használunk a dimenziócsökkentésre 2D ábrázoláshoz, `StandardScaler` előzetes alkalmazásával.

In [ ]:
scaler_pca_part1 = StandardScaler()
X_scaled_part1 = scaler_pca_part1.fit_transform(X_part1)
pca_part1 = PCA(n_components=2, random_state=...)
X_pca_part1 = pca_part1.fit_transform(X_scaled_part1)
plt.figure(...)
plt.scatter(X_pca_part1[:, 0], X_pca_part1[:, 1], c=y_part1, ...)
plt.title('1. Feladat - Generált adathalmaz (PCA után)')

## 1.3 Adat Felosztása és Skálázása

Az eredeti adatokat tanító és teszt halmazra osztjuk (`train_test_split`), majd `StandardScaler`-rel skálázzuk.

In [ ]:
X_train_part1, X_test_part1, y_train_part1, y_test_part1 = train_test_split(
    X_part1, y_part1, test_size=0.3, random_state=..., stratify=y_part1
)
scaler_model_part1 = StandardScaler()
X_train_scaled_part1 = scaler_model_part1.fit_transform(X_train_part1)
X_test_scaled_part1 = scaler_model_part1.transform(X_test_part1)

## 1.4 Modellek Tanítása és Kiértékelése

Definiáljuk, tanítjuk (`fit`) és kiértékeljük (`predict`, `accuracy_score`, `classification_report`, `confusion_matrix`) a Naiv Bayes, SVM és Logisztikus Regresszió modelleket.

In [ ]:
models_part1 = {
    "Naive Bayes": GaussianNB(),
    "Support Vector Machine": SVC(random_state=..., probability=True),
    "Logistic Regression": LogisticRegression(random_state=..., max_iter=1000)
}
for name, model in models_part1.items():
    start_time = time.time()
    model.fit(X_train_scaled_part1, y_train_part1)
    end_time = time.time()
    y_pred_train_part1 = model.predict(X_train_scaled_part1)
    y_pred_test_part1 = model.predict(X_test_scaled_part1)
    accuracy_train_part1 = accuracy_score(y_train_part1, y_pred_train_part1)
    accuracy_test_part1 = accuracy_score(y_test_part1, y_pred_test_part1)
    report_test_part1 = classification_report(y_test_part1, y_pred_test_part1)
    cm_test_part1 = confusion_matrix(y_test_part1, y_pred_test_part1)
    sns.heatmap(cm_test_part1, annot=True, ...)

## 1.5 Eredmények Összefoglalása

Összefoglaljuk a modellek tanítási idejét és pontosságát szövegesen.

In [ ]:
print("\n--- 1. Feladat: Összehasonlítás ---")
print(f"{'Modell':<25} {'Tanítási idő (s)':<20} ...")
for name in results_part1.keys():
    print(f"{name:<25} {training_time:<20.4f} ...")

## 1.6 SVM Hiperparaméter-Hangolás ('C')

`GridSearchCV` segítségével megkeressük az SVM modell optimális `C` paraméterét.

In [ ]:
param_grid_svm_part1 = {'C': [0.01, 0.1, 1, 10, 100]}
svm_cv_part1 = SVC(random_state=..., probability=True)
grid_search_part1 = GridSearchCV(svm_cv_part1, param_grid_svm_part1, cv=5, scoring='accuracy')
grid_search_part1.fit(X_train_scaled_part1, y_train_part1)
best_c = grid_search_part1.best_params_['C']
best_score = grid_search_part1.best_score_
print(f"Legjobb 'C': {best_c}, Legjobb CV pontosság: {best_score:.4f}")
best_svm_part1 = grid_search_part1.best_estimator_
accuracy_svm_best_test_part1 = best_svm_part1.score(X_test_scaled_part1, y_test_part1)
print(f"Legjobb SVM teszt pontosság: {accuracy_svm_best_test_part1:.4f}")

## 1.7 Logisztikus Regresszió Iterációk Hatása

Megvizsgáljuk, hogyan befolyásolja a `max_iter` paraméter a Logisztikus Regresszió pontosságát.

In [ ]:
iterations_part1 = [1, 2, 5, 10, ..., 1000]
for n_iter in iterations_part1:
    lr_iter_part1 = LogisticRegression(random_state=..., max_iter=n_iter)
    lr_iter_part1.fit(X_train_scaled_part1, y_train_part1)
    train_acc = lr_iter_part1.score(X_train_scaled_part1, y_train_part1)
    test_acc = lr_iter_part1.score(X_test_scaled_part1, y_test_part1)
plt.plot(iterations_part1, train_accuracies_lr_part1, ...)
plt.plot(iterations_part1, test_accuracies_lr_part1, ...)

## 1.8 Döntési Határok Vizualizációja

Kirajzoljuk a modellek döntési határait a 2D PCA-val redukált adatokon (`contourf`).

In [ ]:
X_train_pca_part1, X_test_pca_part1, y_train_pca_part1, y_test_pca_part1 = train_test_split(...)
models_pca_part1 = { ... }
xx, yy = np.meshgrid(...)
fig, axes = plt.subplots(...)
for idx, (name, model_pca) in enumerate(models_pca_part1.items()):
    model_pca.fit(X_train_pca_part1, y_train_pca_part1)
    Z = model_pca.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    ax = axes[idx]
    ax.contourf(xx, yy, Z, ...)
    ax.scatter(X_train_pca_part1[:, 0], ..., c=y_train_pca_part1, ...)
    ax.scatter(X_test_pca_part1[:, 0], ..., c=y_test_pca_part1, ...)
    ax.set_title(name)

# 2. Feladat: Osztályozás a Wine Adathalmazon

## 2.1 Adat Betöltése

Betöltjük a Wine adathalmazt URL-ről `pandas.read_csv` segítségével.

In [ ]:
url_part2 = "https://archive.ics.uci.edu/ml/.../wine.data"
column_names_part2 = ['Class', 'Alcohol', ...]
try:
    df_part2 = pd.read_csv(url_part2, header=None, names=column_names_part2)
    print("Wine adathalmaz betöltve.")
except Exception as e:
    print(f"Hiba: {e}")

## 2.2 Adat Vizualizációja (PCA)

PCA-t alkalmazunk a Wine adathalmaz jellemzőire is a 2D vizualizációhoz (`StandardScaler`, `PCA`, `plt.scatter`).

In [ ]:
X_viz_part2 = df_part2.drop('Class', axis=1)
y_viz_part2 = df_part2['Class']
scaler_pca_part2 = StandardScaler()
X_viz_scaled_part2 = scaler_pca_part2.fit_transform(X_viz_part2)
pca_part2 = PCA(n_components=2, random_state=...)
X_pca_part2 = pca_part2.fit_transform(X_viz_scaled_part2)
plt.figure(...)
plt.scatter(X_pca_part2[:, 0], X_pca_part2[:, 1], c=y_viz_part2, ...)
plt.title('2. Feladat - Wine adathalmaz (PCA után)')

## 2.3 Adat Felosztása és Skálázása

Felosztjuk a Wine adatokat tanító és teszt halmazra (`train_test_split`), majd skálázzuk (`StandardScaler`).

In [ ]:
X_part2 = df_part2.drop('Class', axis=1)
y_part2 = df_part2['Class']
X_train_part2, X_test_part2, y_train_part2, y_test_part2 = train_test_split(...)
scaler_model_part2 = StandardScaler()
X_train_scaled_part2 = scaler_model_part2.fit_transform(X_train_part2)
X_test_scaled_part2 = scaler_model_part2.transform(X_test_part2)

## 2.4 Modellek Tanítása és Kiértékelése

Tanítjuk és kiértékeljük a modelleket (NB, SVM, LogReg, k-NN) a Wine adathalmazon.

In [ ]:
models_part2 = {
    "Naive Bayes": GaussianNB(),
    "Support Vector Machine": SVC(random_state=..., probability=True),
    "Logistic Regression": LogisticRegression(random_state=..., max_iter=10000),
    "k-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5)
}
for name, model in models_part2.items():
    model.fit(X_train_scaled_part2, y_train_part2)
    accuracy_score(...)
    classification_report(...)
    confusion_matrix(...)
    sns.heatmap(cm_test_part2, annot=True, ...)

## 2.5 Eredmények Összefoglalása

Összefoglaljuk a Wine adathalmazon elért eredményeket szövegesen.

In [ ]:
print("\n--- 2. Feladat: Összehasonlítás ---")
for name in results_part2.keys():
    print(f"{name:<25} {training_time:<20.4f} ...")

## 2.6 Logisztikus Regresszió Iterációk Hatása

Megvizsgáljuk a Logisztikus Regresszió konvergenciáját a Wine adathalmazon.

In [ ]:
iterations_lr_part2 = [1, 5, ..., 10000]
for n_iter in iterations_lr_part2:
    lr_iter_part2 = LogisticRegression(max_iter=n_iter, ...)
    lr_iter_part2.fit(...)
    train_acc = lr_iter_part2.score(...)
    test_acc = lr_iter_part2.score(...)
plt.plot(iterations_lr_part2, train_accuracies_lr_part2, ...)
plt.plot(iterations_lr_part2, test_accuracies_lr_part2, ...)

## 2.7 SVM Hiperparaméter-Hangolás ('C' és 'gamma')

`GridSearchCV` segítségével megkeressük az SVM modell optimális `C` és `gamma` paramétereit.

In [ ]:
param_grid_svm_part2 = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.1, 1],
    'kernel': ['rbf']
}
svm_cv_part2 = SVC(probability=True)
grid_search_svm_part2 = GridSearchCV(estimator=svm_cv_part2, param_grid=..., cv=5, ...)
grid_search_svm_part2.fit(X_train_scaled_part2, y_train_part2)
best_params = grid_search_svm_part2.best_params_
best_score = grid_search_svm_part2.best_score_
print(f"Legjobb paraméterek: {best_params}, Legjobb CV pontosság: {best_score:.4f}")
best_svm_part2 = grid_search_svm_part2.best_estimator_
accuracy_svm_best_test_part2 = best_svm_part2.score(X_test_scaled_part2, y_test_part2)
print(f"Legjobb SVM teszt pontosság: {accuracy_svm_best_test_part2:.4f}")
sns.heatmap(confusion_matrix(...), annot=True, ...)

## 2.8 SVM Hangolás Vizualizációja (Heatmap)

Heatmap segítségével ábrázoljuk (`seaborn.heatmap`), hogyan változott az SVM keresztvalidációs pontossága a `C` és `gamma` paraméterekre.

In [ ]:
results_df_part2 = pd.DataFrame(grid_search_svm_part2.cv_results_)
results_df_part2 = results_df_part2[['param_C', 'param_gamma', 'mean_test_score']]
try:
    scores_part2 = results_df_part2.pivot(index='param_C', columns='param_gamma', values='mean_test_score')
    sns.heatmap(scores_part2, annot=True, fmt=".4f", cmap="viridis")
    plt.title('2. Feladat - SVM Keresztvalidációs Pontosság (Heatmap)')
except Exception as e:
    pass

## 2.9 Döntési Határok Vizualizációja

Kirajzoljuk a modellek döntési határait a Wine adathalmaz PCA-val redukált változatán (`contourf`).

In [ ]:
X_train_pca_part2, X_test_pca_part2, y_train_pca_part2, y_test_pca_part2 = train_test_split(...)
models_pca_part2 = { ... }
xx_part2, yy_part2 = np.meshgrid(...)
fig, axes = plt.subplots(...)
for idx, (name, model_pca) in enumerate(models_pca_part2.items()):
    model_pca.fit(X_train_pca_part2, y_train_pca_part2)
    Z_part2 = model_pca.predict(...)
    Z_part2 = Z_part2.reshape(xx_part2.shape)
    ax = axes[idx]
    ax.contourf(xx_part2, yy_part2, Z_part2, ...)
    ax.scatter(X_train_pca_part2[:, 0], ...)
    ax.scatter(X_test_pca_part2[:, 0], ...)
    ax.set_title(name)